<a href="https://colab.research.google.com/github/liuyiming0507/My_FEM/blob/master/FEM_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
from scipy.linalg import solve
from scipy.interpolate import griddata
import matplotlib as mpl
# read data
f=open('input_plate_M=50.dat','r')
n=0
Data={}
Num={}
PSN=0
PSS=0
for line in f:
    if 'd0' in line:
        #line.rindex('d0') 
        #count=line.count('d0') #计算substr在S中出现的次数 
        line=line.replace('d','e') 
    if 'D' in line:
        #line.rindex('d0') 
        #count=line.count('d0') #计算substr在S中出现的次数 
        line=line.replace('D','e') 
    Data[n]=l = list(map(str, line.split()))   #将整个line 分隔 
    if 'Plane stress' in line:
        PSS=True
        Num['Plane stress']=n
    if 'Plane strin' in line:
        PSN=True
        Num['Plane strin']=n
    if 'Nodes' in line:
        Num['Nodes']=n
    if 'Element Connectivity' in line:
        Num['Element Connectivity']=n
    if 'Boundary Conditions' in line:
        Num['Boundary Conditions']=n
    if 'End of file' in line:
        Num['End of file']=n
    n=n+1

num_ele=int(Data[Num['Plane stress']+1][0])
num_node=int(Data[Num['Plane stress']+1][1])
young=float(Data[Num['Plane stress']+2][0])
enu=float(Data[Num['Plane stress']+2][1])
nodes = np.zeros(shape=(num_node,2))
ele=np.zeros(shape=(num_ele,3))
bc=np.zeros(shape=(num_node,4))
for i in range(num_node):
    nodes[i]=Data[Num['Nodes']+i+1][1:]
for i in range(num_ele):
    ele[i]=Data[Num['Element Connectivity']+i+1][1:]
ele=ele.astype(int)
for i in range(num_node):
    bc[i]=Data[Num['Boundary Conditions']+i+1][1:]

# stiffness matrix 

#### young's modulus matrix
if PSN==True:
    young = young/(1.0-enu**2)
    enu = enu/(1.0-enu)
E = np.zeros(shape=(3,3),dtype = float)
E[0,0]=young/(1.0-enu**2)
E[0,1]=enu*young/(1.0-enu**2)
E[1,0]=enu*young/(1.0-enu**2)
E[1,1]=young/(1.0-enu**2)
E[2,2]=(1.0-enu)*young/2*(1.0-enu**2)

#### area of element
A={} #area of element
for i in range(num_ele):
    aa=np.array([[1.0, nodes[ele[i,0]-1,0], nodes[ele[i,0]-1,1]],
        [1.0, nodes[ele[i,1]-1,0], nodes[ele[i,1]-1,1]],
        [1.0, nodes[ele[i,2]-1,0], nodes[ele[i,2]-1,1]]])
    A[i]=round(0.5*np.linalg.det(aa),14)
    
#### B matrix
B={} #B为一个记录各个单元B matrix的字典
for i in range(num_ele):
    X=[nodes[ele[i,0]-1,0], nodes[ele[i,1]-1,0], nodes[ele[i,2]-1,0]]
    Y=[nodes[ele[i,0]-1,1], nodes[ele[i,1]-1,1], nodes[ele[i,2]-1,1]]
    bb=np.array([[Y[1]-Y[2], 0, Y[2]-Y[0], 0, Y[0]-Y[1], 0], 
                  [0, X[2]-X[1], 0, X[0]-X[2], 0, X[1]-X[0]], 
                  [X[2]-X[1], Y[1]-Y[2], X[0]-X[2], Y[2]-Y[0], X[1]-X[0], Y[0]-Y[1]]])
    B[i]=bb/(2*A[i])
#### element stiffness matrix
K={} #K为一个记录各个单元的单元刚度矩阵的字典，检索号为单元号K[单元号]
for i in range(num_ele):
    kk=np.dot(np.dot(B[i].T,E),B[i])
    K[i]=A[i]*kk
#### The overall stiffness matrix 
cplt_K=np.zeros([2*num_node,2*num_node])
for i in range(num_ele):
    for l in range(3):
        for m in range(3):
            a=2*(ele[i,l]-1)
            b=2*(ele[i,m]-1)
            cplt_K[a,b]=cplt_K[a,b]+K[i][2*l,2*m]
            cplt_K[a,b+1]=cplt_K[a,b+1]+K[i][2*l,2*m+1]
            cplt_K[a+1,b]=cplt_K[a+1,b]+K[i][2*l+1,2*m]
            cplt_K[a+1,b+1]=cplt_K[a+1,b+1]+K[i][2*l+1,2*m+1]

# Operate BC
dsplm=[]
force=[]
for i in range(num_node):
    if bc[i,0]==1:
        if bc[i,1]==0:
            dsplm.append(2*i)
    else:
        force.append(bc[i,1])
    if bc[i,2]==1:
        if bc[i,3]==0:
            dsplm.append(2*i+1)
    else:
        force.append(bc[i,3])  

# solve
df1= DataFrame(cplt_K)
df1=df1.drop(dsplm)
df1=df1.drop(dsplm, axis=1)
cut_K=df1.values
part_U=solve(cut_K,force)
for i in range(len(part_U)):
    part_U[i]=round(part_U[i],14)
U=np.zeros([2*num_node])
nn=0
for i in range(2*num_node):
    if i in dsplm:
        U[i]=0
    else:
        U[i]=part_U[nn]
        nn=nn+1

F=np.dot(cplt_K,U)
for i in range(len(F)):
    F[i]=round(F[i],14)

ele_U={} 
for i in range(num_ele):
    ele_dof=np.array([ U[2*(ele[i,0]-1)] , U[2*(ele[i,0]-1)+1] , 
                       U[2*(ele[i,1]-1)] , U[2*(ele[i,1]-1)+1] ,
                       U[2*(ele[i,2]-1)] , U[2*(ele[i,2]-1)+1] ])
    ele_U[i]=ele_dof
    
strain={}
stress={}
for i in range(num_ele):
    strain[i]=np.dot(B[i],ele_U[i])
    for j in range(3):
        strain[i][j]=round(strain[i][j],14)
    stress[i]=np.dot(E,strain[i])
    for k in range(3):
        stress[i][k]=round(stress[i][k],14)
#print(strain)
#print(stress)
node_strain=np.zeros([num_node,3])
node_stress=np.zeros([num_node,3])
weight_num=np.zeros([num_node])
for i in range(num_node):
    for j in range(num_ele):
        if i+1 in list(ele[j]):
            weight_num[i]=weight_num[i]+1
            node_strain[i,:]= node_strain[i,:]+strain[j]
            node_stress[i,:]= node_stress[i,:]+stress[j]
for i in range(num_node):
    node_strain[i,:]=node_strain[i,:]/weight_num[i]
    node_stress[i,:]=node_stress[i,:]/weight_num[i]
#print(node_strain)
#print(node_stress)
U=U.reshape(num_node,2)
#print(U)
F=F.reshape(num_node,2)
#print(F)

# visulization
X,Y = np.meshgrid(nodes[:,0],nodes[:,1])
fig = plt.figure()
grid_z0 = griddata(nodes, F[:,0], (X, Y), method='linear')
print(grid_z0)
surf1 = plt.contourf(X,Y,grid_z0)
#plt.pcolor(X, Y, grid_z0,  vmin=-0.05, vmax=0)
plt.colorbar(surf1,shrink=1)

FileNotFoundError: ignored